<a href="https://www.kaggle.com/code/noahnguyen08/customer-churn-classification-and-network-analysis?scriptVersionId=164925678" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer-churn-data-infinite-investment-systems/account_data_info.csv
/kaggle/input/customer-churn-data-infinite-investment-systems/history.csv
/kaggle/input/customer-churn-data-infinite-investment-systems/test.csv


In [ ]:
!pip install kmodes
!pip install networkx
!pip install prince

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import prince                 # MCA

import datetime
import category_encoders as ce

from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from kmodes.kmodes import KModes
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.metrics import roc_curve, auc

from PIL import Image

# from kneed import KneeLocator
%matplotlib inline

## Loading data

In [ ]:
df = pd.read_csv("/kaggle/input/customer-churn-data-infinite-investment-systems/history.csv")
test = pd.read_csv("/kaggle/input/customer-churn-data-infinite-investment-systems/test.csv")

In [ ]:
data_info = pd.read_csv("/kaggle/input/customer-churn-data-infinite-investment-systems/account_data_info.csv", encoding = "latin-1")

In [ ]:
df_copy = df[:]        # make a copy for customer churn data
test_copy = test[:]    # make a copy for test data

In [ ]:
data_info_copy = data_info[:]

## Data Processing

Description of each feature

In [ ]:
data_info_copy.head(10)

### Dealing with missing values

There are missing data in the description of some features. We'd proceed imputing these missing values with an empty string.

In [ ]:
data_info_copy['Description'] = data_info_copy['Description'].fillna(' ')   # imputing missing data with empty string
data_info_copy['Field Values'] = data_info_copy['Field Values'].fillna(' ')

In [ ]:
data_info_copy['Columns'][1]

In [ ]:
data_info_copy.head(5)

In [ ]:
pd.set_option('display.max_columns', None)    # show all columns

In [ ]:
df_copy.columns[0]

We observe that the number of columns in the data description and the customer data is not the same. We'd check what are the columns that don't exist in the customer data.

In [ ]:
data_info_lst = []
customer_data_lst = []
for val in range(len(data_info_copy['Columns'])):
  data_info_lst.append(data_info_copy['Columns'][val])

for col in range(len(df_copy.columns)):
  customer_data_lst.append(df_copy.columns[col])

In [ ]:
print(data_info_lst)

In [ ]:
print(customer_data_lst)

In [ ]:
info_DNE = []
for i in range(len(data_info_lst)):
  if data_info_lst[i] not in customer_data_lst:
    info_DNE.append(data_info_lst[i])

In [ ]:
info_DNE

These are the columns that didn't appear on the customer data.

In [ ]:
df_copy['country_code'].isnull().value_counts()

In [ ]:
df_copy['currency_code'].isnull().value_counts()

`country_code` and `currency_code` have the same purpose, let's say your currency code is CAD, it's likely that you are from Canada, thus the `country_code` may not contribute any useful information. Thus we'd drop this column.

In [ ]:
df_copy = df_copy.drop('country_code', axis=1)

In [ ]:
df_copy['cashflows_custody_fee'].isnull().value_counts()

As all the records in `cashflows_custody_fee` are missing, we'd just drop this column as well.

In [ ]:
df_copy = df_copy.drop('cashflows_custody_fee', axis=1)

In [ ]:
df_copy['esir_number'].isnull().value_counts()

In [ ]:
df_copy = df_copy.drop('esir_number', axis=1)

In [ ]:
def missing_data_per_col(data):
  '''
    Show the number of missing values
    in each column

    input: data
  '''
  null_count = []
  missing_data = {}
  for col in data.columns:
    missing_data[col] = [data[col].isna().sum()]
  missing_df = pd.DataFrame(missing_data)
  missing_df = missing_df.rename(columns = {0: "Number of missing values"})
  return missing_df


In [ ]:
missing_data_df = missing_data_per_col(df_copy)

In [ ]:
missing_data_df

In [ ]:
len(missing_data_df.columns)

### Analyzing the contribution of each feature

#### Categorical features that have too many categories -> K-modes clustering

In [ ]:
print("Number of categories in class_id: ", len(df_copy['class_id'].unique()))
print("Number of categories in terminal_code: ", len(df_copy['terminal_code'].unique()))
print("Number of categories in sss_agent: ", len(df_copy['sss_agent'].unique()))
print("Number of categories in deceased_fair_market_value: ", len(df_copy['deceased_fair_market_value'].unique()))
print("Number of categories in risk_tolerance: ", len(df_copy['risk_tolerance'].unique()))
print("Number of categories in investment_objective: ", len(df_copy['investment_objective'].unique()))
print("Number of categories in last_maintenance_user: ", len(df_copy['last_maintenance_user'].unique()))
print("Number of categories in special_fee_code: ", len(df_copy['special_fee_code'].unique()))
print("Number of categories in retail_last_maintenance_user: ", len(df_copy['retail_last_maintenance_user'].unique()))
print(len(df_copy['retail_last_maintenance_user'].unique()))

Drop `rep_commission_rate`, `dup_trip_quad_code`, `portfolio_cost_method`, `portfolio_name_address_option` columns as they only have one single value.

In [ ]:
# df_copy = df_copy.drop('rep_commission_rate', axis=1)
# df_copy = df_copy.drop('dup_trip_quad_code', axis=1)
# df_copy = df_copy.drop('portfolio_cost_method', axis=1)
# df_copy = df_copy.drop('portfolio_name_address_option', axis=1)
# df_copy = df_copy.drop('portfolio_summary_option', axis=1)
# df_copy = df_copy.drop('interactive_portfolio_code', axis=1)
# df_copy = df_copy.drop('non_plan_book_value_flag', axis=1)

In [ ]:
df_copy

In [ ]:
df_copy_time_based_cols = df_copy[['inception_date', 'last_update_date', 'last_maintenance_time', 'rrif_original_date', 'plan_effective_date', 'plan_end_date', 'last_trade_date', 'inserted_at', 'updated_at', 'retail_last_maintenance_time']]
# for time-based columns, create dummy variables -> year, month, day and convert them to int dtype


df_copy_bool_based_cols = df_copy[['is_registered', 'is_active', 'net_of_fees', 'fee_paid_separately', 'custody_fee_withdrawal', 'is_fee_exempt', 'include_client_consolidation', 'use_client_address',
                                   'is_spousal', 'is_arp_locked', 'is_midwest_clearing_account', 'use_hand_delivery', 'use_mail', 'share_name_address_to_issuer', 'shareholder_instructions_received',
                                   'rrsp_limit_reached', 'is_portfolio_account', 'has_no_min_commission', 'is_tms_eligible', 'is_agent_bbs_participant', 'is_parameters_account', 'is_spousal_transfer',
                                   'spousal_age_flag', 'has_multiple_name', 'discretionary_trading_authorized', 'receive_general_mailings', 'has_discrete_auth', 'is_non_objecting_beneficial_owner',
                                   'is_objecting_to_disclose_info', 'consent_to_pay_for_mail', 'consent_to_email_delivery', 'has_received_instruction', 'is_broker_account', 'is_inventory_account',
                                   'is_gl_account', 'is_control_account', 'is_extract_eligible', 'is_pledged', 'is_resp', 'use_original_date_for_payment_calc', 'is_family_resp', 'is_hrdc_resp',
                                   'is_plan_grandfathered', 'is_olob', 'visible_in_reports']]

# for bool-based columns, convert to binary (0 and 1)

df_copy_categorical_based_cols = df_copy[['type_code', 'currency_code', 'class_id', 'debit_code', 'contract_type', 'branch', 'credit_limit_type', 'retail_plan', 'arp_pension_origin', 'language_code', 'dividend_confirm_code',
                                          'sss_location', 'options_trading_type', 'sss_type', 'sss_agent', 'rep_commission_override', 'interest_dividend_conversion_type', 'guarantee_gtor_type', 'terminal_code',
                                          'deceased_fair_market_value', 'target_grantor_grantee_flag', 'iso_funds_code', 'shareholder_language', 'special_tag', 'conjunction', 'title', 'function_code', 'tms_settlement_location',
                                          'portfolio_report_option', 'loan_limit_override', 'mailing_consent', 'risk_tolerance', 'investment_objective', 'last_maintenance_user', 'special_fee_code', 'non_calendar_year_end',
                                          'number_of_beneficiaries', 'resp_specimen_plan', 'retail_last_maintenance_user']]

# for categorical-based columns, perform clustering (for features that contain a lot of categories)

# the number of clusters will be decided by elbow method (or domain knowledge)

Check if there is any missing columns after assigning columns into variables

In [ ]:
count_missing_col = 0
for i in range(len(df_copy_time_based_cols.columns)):
  if df_copy_time_based_cols.columns[i] not in df_copy.columns:
    count_missing_col += 1

for i in range(len(df_copy_bool_based_cols.columns)):
  if df_copy_bool_based_cols.columns[i] not in df_copy.columns:
    count_missing_col += 1

for i in range(len(df_copy_categorical_based_cols.columns)):
  if df_copy_categorical_based_cols.columns[i] not in df_copy.columns:
    count_missing_col += 1
print(count_missing_col)

#### Check the null values in each feature

In [ ]:
nan_count = df_copy.isna().sum()

In [ ]:
nan_count.sort_values(axis=0).value_counts().sort_index()

We observe that there are 39 columns that contain no missing values. We will perform clustering on those columns

In [ ]:
no_nan_cols = df_copy[list(nan_count[nan_count == 0].index)]

We have to drop the target variable (label: churn or no churn)

In [ ]:
no_nan_cols = no_nan_cols.drop('label', axis=1)

In [ ]:
no_nan_cols = no_nan_cols.drop('id', axis=1)

In [ ]:
no_nan_cols

In [ ]:
no_nan_cols_no_time_series = no_nan_cols.drop(['inserted_at', 'updated_at'], axis=1)

In [ ]:
no_nan_cols_no_time_series_dummies = pd.get_dummies(no_nan_cols_no_time_series)   # exclude time-series features

In [ ]:
no_nan_cols_no_time_series_dummies

#### Feature analysis

If some of features contain duplicate information (they convey the same information to our dataset), then it's best to visualize which one they are (how they variate), and decide to drop such features (feature selection).

In [ ]:
# correlation between features
corr_matrix = no_nan_cols_no_time_series_dummies.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot = True, cmap = 'YlGnBu')

The above is the Corr[X, Y] where X and Y represent the pair of any 2 features that we examine by correlation heatmap

In [ ]:
corr_matrix

High correlation presented: As the correlation heatmap shows, there are a few features that share a high correlation level to another, suggesting that they have a linear relationship with each other
* Linear relationship means that one feature is the linear transformation of the other, causing multicollinearity (potentially duplicate) information. Thus we would consider drop one of them.
* We will set a threshold of 0.5 for the correlation level; any pair of features whose level of correlation exceeds the threshold will be examined and excluded by domain knowledge.

In [ ]:
corr_thresh = 0.5

corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j] > corr_thresh):
            corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))

for pair in corr_pairs:
    print(f"Features '{pair[0]}' and '{pair[1]}' have correlation of: '{pair[2]}'")

We observe that there are some features that have high correlation with many other features. We will drop such features. (Talk a little bit about the domain knowledge before dropping those features)

In [ ]:
df_copy = df_copy.drop(['include_client_consolidation', 'use_client_address', 'share_name_address_to_issuer',
                        'receive_general_mailings', 'is_non_objecting_beneficial_owner', 'is_olob', 'use_mail'], axis=1)

no_nan_cols_no_time_series_dummies = no_nan_cols_no_time_series_dummies.drop(['include_client_consolidation_t', 'include_client_consolidation_f',
                                                              'use_client_address_t', 'use_client_address_f', 'share_name_address_to_issuer_t',
                                                              'share_name_address_to_issuer_f', 'receive_general_mailings_t', 
                                                              'receive_general_mailings_f', 'is_non_objecting_beneficial_owner_t',
                                                              'is_non_objecting_beneficial_owner_f', 'is_olob_t', 'is_olob_f', 
                                                              'use_mail_t', 'use_mail_f'], axis=1)

#### Feature importances

Looks like there are still a lot of features. We will continue examine each feature importance to its target variable. We will perform a simple Random Forest model on our non-null data to decide what features to keep, and what to drop.

In [ ]:
# extract the label used for random forest
rf_label = df_copy['label']

In [ ]:
rf_label.unique()

In [ ]:
rf_label = rf_label.apply(lambda x: 1 if x == "Churn" else 0)

In [ ]:
rf_label

In [ ]:
no_nan_data_model = RandomForestClassifier()
no_nan_data_model.fit(no_nan_cols_no_time_series_dummies, rf_label)

In [ ]:
plt.figure(figsize=(20, 12))
feature_importances = pd.Series(no_nan_data_model.feature_importances_, index = no_nan_cols_no_time_series_dummies.columns)
feature_importances.nlargest(10).plot(kind='barh')
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Top 10 Feature Importances')
plt.show()

This simple Random Forest Classifier did a decent job. As we analyze using our domain knowledge, if a client is **not visible in reports**, it may indicate that the client has been inactive for a long time, leading to the churn decision of that client.

#### Evaluate the importance of each feature to the target

In [ ]:
auc_scores = []
roc_curves = []

for feature in no_nan_cols_no_time_series_dummies.columns:
    _feature = no_nan_cols_no_time_series_dummies[[feature]]
    no_nan_data_model.fit(_feature, rf_label)
    
    y_pred = no_nan_data_model.predict_proba(_feature)[:, 1]
    
    fpr, tpr, _ = roc_curve(rf_label, y_pred)
    roc_curves.append((fpr, tpr))
    
    auc_score = auc(fpr, tpr)
    auc_scores.append(auc_score)

In [ ]:
plt.figure(figsize=(20, 12))
for fpr, tpr in roc_curves:
    plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for each feature')
plt.legend(no_nan_cols_no_time_series_dummies.columns, loc='lower right', fontsize='x-small')
plt.show()

In [ ]:
# print AUC score for each feature
for i, feature in enumerate(no_nan_cols_no_time_series_dummies.columns):
    print(f"Feature '{feature}': AUC = {auc_scores[i]}")

Question: Should we drop features based solely on AUC score of each feature?

In [ ]:
#### Rule of thumb for "good" AUC score

# Each feature with an AUC score above 0.7 would be considered acceptable, therefore any features with AUC score < 0.7 would be considered for removing.

# df_copy = df_copy.drop(['currency_code', 'is_active', 'net_of_fees', 'fee_paid_separately', 'custody_fee_withdrawal', 'is_fee_exempt', 'use_hand_delivery', 'shareholder_instructions_received', 
#                         'rrsp_limit_reached', 'is_portfolio_account', 'has_no_min_commission', 'is_tms_eligible', 'is_agent_bbs_participant', 'is_parameters_account', 'is_spousal_transfer', 
#                         'spousal_age_flag', 'has_multiple_name', 'is_objecting_to_disclose_info', 'consent_to_pay_for_mail', 'consent_to_email_delivery', 'has_received_instruction', 'is_broker_account', 
#                         'is_inventory_account', 'is_gl_account', 'is_control_account', 'is_extract_eligible'], axis=1)

In [ ]:
# no_nan_cols_no_time_series_dummies = no_nan_cols_no_time_series_dummies.drop(['currency_code_CAD', 'currency_code_USD', 'is_active_f', 'is_active_t', 'net_of_fees_f',
#                                                                               'fee_paid_separately_f', 'custody_fee_withdrawal_f', 'is_fee_exempt_f', 
#                                                                               'use_hand_delivery_f', 'use_hand_delivery_t', 'shareholder_instructions_received_f', 'shareholder_instructions_received_t',
#                                                                               'rrsp_limit_reached_f', 'is_portfolio_account_f', 'has_no_min_commission_f',
#                                                                               'is_tms_eligible_f', 'is_tms_eligible_t', 'is_agent_bbs_participant_f', 'is_agent_bbs_participant_t', 'is_parameters_account_f',
#                                                                               'is_spousal_transfer_f', 'is_spousal_transfer_t', 'spousal_age_flag_f', 'has_multiple_name_f', 'has_multiple_name_t',
#                                                                               'is_objecting_to_disclose_info_f', 'is_objecting_to_disclose_info_t', 'consent_to_pay_for_mail_f', 'consent_to_pay_for_mail_t',
#                                                                               'consent_to_email_delivery_f', 'consent_to_email_delivery_t', 'has_received_instruction_f', 'has_received_instruction_t',
#                                                                               'is_broker_account_f', 'is_inventory_account_f', 'is_inventory_account_t', 'is_gl_account_f',
#                                                                               'is_control_account_f', 'is_extract_eligible_f', 'is_extract_eligible_t'], axis=1)

In [ ]:
no_nan_cols_no_time_series_dummies

As clustering on the above data would cause huge time complexity and resources, we have to perform some dimensionality reduction techniques such as PCA to extract new set of features that bring most variance (retain the original information) of our dataset.

### PCA

In [ ]:
no_nan_cols_dummies_train = np.array(no_nan_cols_no_time_series_dummies).astype(np.float32)

In [ ]:
no_nan_cols_dummies_train

In [ ]:
cov_mat = np.cov(no_nan_cols_dummies_train.T)
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)

print(f"Eigenvalues: \n {eigen_vals}")
print(f"Eigenvalues shape: {eigen_vals.shape}")

In [ ]:
def explain_variance(eigenvalues):
    '''
        Plot how many PCA axes needed to explain most variance
        from the original feature space of our dataset
    '''
    var_explained = []
    total = sum(eigenvalues)
    for i in sorted(eigenvalues, reverse=True):
        var = i / total
        var_explained.append(var)
    cummulative_var_explained = np.cumsum(var_explained)
    plt.bar(range(1, len(eigenvalues)+1), var_explained, alpha=0.5, align='center', label='Individual explained variance ratio')
    plt.step(range(1, len(eigenvalues)+1), cummulative_var_explained, where='mid', label='Cummulative explained ratio')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal component index')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

In [ ]:
explain_variance(eigen_vals)

As we can see, with the number of PCA axes of 10 axes, they already explained almost 100% of the variance of the original data. We will pick 10 as the number of clusters for our SBM disussed later on.

Since the objective is to classify whether a user churned or didn't churn from the service, so we have to drop the plan_end_date column.

In [ ]:
df_copy = df_copy.drop('plan_end_date', axis=1)

In [ ]:
df_copy

We have extracted 10 PCA axes as the ideal number of clusters for our clustering algorithm for the non-null dataset.

In [ ]:
# cluster the data with the number of clusters = 10
optimal_num_clusters = 10
kmode_optimal = KModes(n_clusters = optimal_num_clusters, init = 'Cao', n_init = 1, verbose = 1)          # applied K-modes clustering for categorical data
clusters = kmode_optimal.fit_predict(no_nan_cols_no_time_series_dummies)

In [ ]:
np.unique(clusters)             # 10 unique clusters

In [ ]:
df_copy['Clusters'] = clusters

In [ ]:
df_copy

In [ ]:
# check if all records per cluster add up to the original number of records
rec_count = 0
for i in range(len(np.unique(clusters))):
    rec_count = rec_count + len(df_copy.loc[df_copy['Clusters'] == i])

In [ ]:
rec_count

### Filling missing values
Each record in our original dataset (containing null values) has been assigned to each of the 10 clusters. We proceed with filling null values by the mode of each column corresponding to each null value and the cluster number.

In [ ]:
# if the cluster is 0, then loop over every column within each record corresponding to cluster 0. The same thing with other clusters
def fill_nan_by_mode(data, cluster_col):
    '''
        Fill null values by the mode of each column
        corresponding to each null value and cluster number
        
        data: DataFrame
        cluster_col: name of the cluster column
    '''
    for cluster_num in data[cluster_col].unique():
        cluster_data = data[data[cluster_col] == cluster_num]
        
        # fill null values with mode for each cluster
        cluster_mode = cluster_data.mode().iloc[0]       # select the first row and calculate the mode for each column in DataFrame, the result is that row containing the mode value of each column
        data.loc[data[cluster_col] == cluster_num] = data.loc[data[cluster_col] == cluster_num].fillna(cluster_mode)
    return data

In [ ]:
# # experimenting filling missing values by mode without considering the cluster
# def fill_nan_by_mode_no_clusters(data, cluster_col):
#     for col in data.columns:
#         cluster_mode = data[col].mode()[0]
#         data[col] = data[col].fillna(cluster_mode)
#     return data

In [ ]:
df_copy = fill_nan_by_mode(df_copy, 'Clusters')

In [ ]:
# df_copy_3 = fill_nan_by_mode_no_clusters(df_copy_3, 'Clusters')

In [ ]:
# df_copy_3   # this only fill the missing values of each column with its mode, which will cause bias for our model -> the method of filling missing values with mode of the column based on cluster works better

In [ ]:
# print the number of null values for each column for df_copy2
for col in df_copy.columns:
    print(f"'{col}' nulls: {df_copy[col].isna().sum()}")

There are a lot of columns missing 7427 values, so we suspect that there are 7427 rows that contain missing values in those columns. Observe that there is 'tms_settlement_location' containing 186399 null values which take up a lot of our data, so we might consider dropping that column. After checking the description of this column, the settlement location doesn't really determine the churn behavior of a client (there are features that have stronger relationship to the churn behavior), thus we drop 'tms_settlement_location'. Also, for columns that contain less than 100000 null values, we will drop those null values since they aren't a huge part of our dataset. We perform these methods, then rerun the loop above to check for null values in each column.

In [ ]:
for col in df_copy.columns:
    if df_copy[col].isna().sum() < 100000:
        df_copy.dropna(subset=[col], inplace=True)

In [ ]:
df_copy = df_copy.drop('tms_settlement_location', axis=1)

In [ ]:
df_copy          # clean df_copy, dataset contains 0 null values

In [ ]:
# print the number of null values for each column for df_copy2
for col in df_copy.columns:
    print(f"'{col}' nulls: {df_copy[col].isna().sum()}")

Each client has a unique ID, but since the default index column in pandas contains unique indices as well for each record (client), we should drop the 'id' column to reduce the dimension of the encoded dataset later on, as every value in the ID column is unique so there will be many categories when decoding.

In [ ]:
df_copy = df_copy.drop('id', axis=1)

In [ ]:
df_copy.reset_index(drop=True, inplace=True)

In [ ]:
df_copy = df_copy.drop(['plan_effective_date'], axis=1)                                  # drop plan_effective_date, since inception_date dates come first
df_copy_time_based_cols = df_copy_time_based_cols.drop(['plan_end_date', 'plan_effective_date'], axis=1)

In [ ]:
# convert date time to just year month day
for col in df_copy_time_based_cols.columns:
    df_copy[col] = pd.to_datetime(df_copy[col])

In [ ]:
df_copy

In [ ]:
# get the label column to assign to target outcome later on
label_ = df_copy['label']

In [ ]:
df_copy = df_copy.drop(['label', 'Clusters'], axis=1)

In [ ]:
# check the number of categories per column
total_categories = 0
for col in df_copy.columns:
    print(f"Number of categories in '{col}': {len(df_copy[col].unique())}")
    total_categories += len(df_copy[col].unique())
print(f"Total number of categories: {total_categories}")

### Encoding dataset: Binary encoding
There are time-series features that contain a lot of different categories (time, hour). If we chose to perform one-hot encoding, it may cause a very large dataset and there will not be enough memory. Binary Encoding is the alternative.

In [ ]:
binary_encoder = ce.BinaryEncoder(cols=list(df_copy.columns), return_df=True)

In [ ]:
df_copy_encoded = binary_encoder.fit_transform(df_copy)
df_copy_encoded    # encode df_copy

In [ ]:
# correlation between features
corr_matrix_ = df_copy_encoded.corr()
corr_matrix_

### Correlations between pairs of features that exceed the correlation threshold

In [ ]:
corr_thresh_ = 0.7

corr_pairs_ = []
for i in range(len(corr_matrix_.columns)):
    for j in range(i+1, len(corr_matrix_.columns)):
        if abs(corr_matrix_.iloc[i, j] > corr_thresh_):
            corr_pairs_.append((corr_matrix_.columns[i], corr_matrix_.columns[j], corr_matrix_.iloc[i, j]))

for pair in corr_pairs_:
    print(f"Features '{pair[0]}' and '{pair[1]}' have correlation of: '{pair[2]}'")

In [ ]:
df_copy_encoded

In [ ]:
corr_thresh_ = 0.7

corr_pairs_ = []
corr_hash = {}

for i in range(len(corr_matrix_.columns)):
    for j in range(i+1, len(corr_matrix_.columns)):
        if abs(corr_matrix_.iloc[i, j] > corr_thresh_):
            corr_pairs_.append((corr_matrix_.columns[i], corr_matrix_.columns[j], corr_matrix_.iloc[i, j]))
            # if a feature appears at least 2 times when having high correlation with other features -> drop such features
            if corr_matrix_.columns[i] in corr_hash:
                df_copy_encoded = df_copy_encoded.drop(corr_matrix_.columns[i], axis=1)
                break
            else:
                corr_hash[corr_matrix_.columns[i]] = corr_matrix_.iloc[i, j]

In [ ]:
df_copy_encoded

In [ ]:
# get the correlation between pair of features after excluding those with high correlation

corr_thresh_start = 0.5
corr_thresh_end = 0.7

corr_pairs_ = []
for i in range(len(corr_matrix_.columns)):
    for j in range(i+1, len(corr_matrix_.columns)):
        if abs(corr_matrix_.iloc[i, j] >= corr_thresh_start) and abs(corr_matrix_.iloc[i, j] <= corr_thresh_end):
            corr_pairs_.append((corr_matrix_.columns[i], corr_matrix_.columns[j], corr_matrix_.iloc[i, j]))

for pair in corr_pairs_:
    print(f"Features '{pair[0]}' and '{pair[1]}' have correlation of: '{pair[2]}'")

In [ ]:
# get the list of features which have moderate correlation with other features
feat_corr_moderate = []
for i in range(len(corr_pairs_)):
    for j in range(len(corr_pairs_[i][:2])):
        if corr_pairs_[i][j] not in feat_corr_moderate:
            feat_corr_moderate.append(corr_pairs_[i][j])

In [ ]:
feat_corr_moderate

### Relationship between each feature with moderate correlation and the target outcome

In [ ]:
df_copy['type_code'].unique()

In [ ]:
df_copy['number_of_beneficiaries'].unique()

### MCA: Reduce the dimension for categorical data

In [ ]:
# init MCA
mca = prince.MCA(n_components=50, n_iter=10, copy=True, check_input=True, engine='sklearn', random_state=42)

The number of components n_components are determined by experimenting how many components are enough to capture a good percentage of variance of our data.

In [ ]:
# fit MCA to our encoded data
mca.fit(df_copy_encoded)

In [ ]:
# # transform df_copy_encoded to the MCA version
# df_copy_encoded_mca = mca.transform(df_copy_encoded)

In [ ]:
# extract the eigenvalues -> see the cummulative variance explained -> pick the number of components that explain enough variance for our data
mca.eigenvalues_summary

After trials and failures, 50 components and 10 iterations capture more than 50% of the variance of our encoded data. This implies that 50 is a good number of features that still capture enough variance for our data.

In [ ]:
# # visualization
# mca.plot(df_copy_encoded, x_component=0, y_component=1, show_column_markers=True, show_row_markers=True, show_column_labels=False, show_row_labels=False)

In [ ]:
# # visualization
# fig, ax = plt.subplots(figsize=(10, 7))
# ax.scatter(df_copy_encoded_mca_[0], df_copy_encoded_mca_[1], c='blue', edgecolors='k', alpha=0.5)
# ax.set_xlabel('Dimension 1')
# ax.set_ylabel('Dimension 2')
# ax.set_title('MCA results')
# plt.axhline(0, color='grey', lw=1)
# plt.axvline(0, color='grey', lw=1)
# plt.grid(True)
# plt.show()

## Preparing data for model training

In [ ]:
def binary_to_numeric(value):
    '''
        Convert "Churn" to 1
        and "No Churn" to 0
    '''
    if value == "Churn":
        return 1
    elif value == "No Churn":
        return 0
    else:
        return None

In [ ]:
# target outcome
y_ = label_.apply(binary_to_numeric)

In [ ]:
y_

In [ ]:
df_copy_encoded['label'] = y_

In [ ]:
df_copy_encoded

### Relationship between features with moderate correlation and the target outcome 'label'

In [ ]:
# @title
plt.figure(figsize=(80, 12))  # Adjust the figure size as needed
sns.set(style="whitegrid")

plt.subplot(2, 2, 1)
sns.histplot(x=df_copy['type_code'], hue=df_copy_encoded['label'], bins=30, kde=False, color='blue').set(title='Relationship between Outcome and type_code')

Recall that label of 1 is Churned, 0 for Not Churned. Looks like most clients with type_code of CASH SWEEP, CASH, and MRGN churned from the service

In [ ]:
# @title
plt.figure(figsize=(20, 12))  # Adjust the figure size as needed
sns.set(style="whitegrid")

plt.subplot(2, 2, 1)
sns.histplot(x=df_copy['number_of_beneficiaries'], hue=df_copy_encoded['label'], bins=30, kde=False, color='blue').set(title='Relationship between Outcome and number of beneficiaries')

Most of the clients doesnt' have any beneficiaries, and we can see that most clients churned.

In [ ]:
df_copy_train, df_ = train_test_split(df_copy_encoded, test_size=0.40, random_state=1)
df_copy_cv, df_copy_test = train_test_split(df_, test_size=0.50, random_state=1)

del df_

X_train = df_copy_train.drop('label', axis=1)
y_train = df_copy_train['label']

X_cv = df_copy_cv.drop('label', axis=1)
y_cv = df_copy_cv['label']

X_test = df_copy_test.drop('label', axis=1)
y_test = df_copy_test['label']

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_cv shape: {X_cv.shape}")
print(f"y_cv shape: {y_cv.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
X_train = np.array(X_train).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)

X_cv = np.array(X_cv).astype(np.float32)
y_cv = np.array(y_cv).astype(np.float32)

X_test = np.array(X_test).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)

## Modeling:
After getting exhausted with the data processing part, we are now ready to develop a model to discover the underlying relationship and behavior among customers, helping recommend marketing strategies to maximize the likelihood of staying.

#### Motivation
We came up with the idea that customers, who may not know each other, will somehow relate to each other based on psychology, for example, and that could determine the churn/no churn behavior of each client. Thus, we treat the customer data as a network, and we will be performing a Stochastic Block Model (SBM) and statistical methods (potentially applying Bernoulli distributions, inhomogeneous Poisson process) to understand the underlying relationship among clients within each clusters, and clients among clusters.

### XGBoost
We start by training our data on a simple XGBoost to predict and classify each client to Churn/No Churn, and use the result to extract the groups clients based on behavior (for example, "Not Churned, but are likely to churn: At Risk", "Churned, but loved a specific product", "Not Churned, and enjoy the overall service").

### Stochastic Block Model (SBM)

#### Predictive Modeling (for Churn classification and Marketing Strategies to Maximize Likelihood of Staying)
SBM can predict missing or future connections in networks, and forecast node attributes based on their network position and block memberships.

#### Network Analysis
It's a statistical framework used to analyze networks by partitioning nodes (customers) into blocks (clusters) based on theier connectivity (relationship) patterns. It assumes that nodes within the same block have similar interaction probabilities, while interaction probabilities between nodes in different blocks are governed by different parameters. In the context of SBM, the adjacency matrix is analyzed to identify the blocks or segments of customers. These blocks represent groups of customers who interact with each other more frequently than with customers outside their block. The identification of blocks can be achieved through various clustering algorithms that are designed to work with SBM. SBM estimates the probabilities of interactions between and within these blocks
* A high probability of interaction within a block but low between blocks might indicate a strong community effect that can be significant for marketing strategies

* Notice there are 2 blocks that don't have mutual connection, because the block label is the negation of each other.
* There are blocks that have connections ('like apples hate grapes' and 'like oranges hate grapes'), because these 2 blocks both hate grapes
* Generally, the probability of edges (connections) between nodes within the same block is higher than that of between blocks, which makes intuitive sense
* Each edge represents the likelihood of an edge (connection) existing between pairs of nodes in the network
* The edge probabilities can vary depending on whether the nodes belong to the same block (within-block edges) or different block (between-block edges)

##### Within-block edge probability
* Within-block edge probabilities are denoted by p_in

##### Between-block edge probability
* Between-block edge probabilities are denoted by p_out

##### Probability distribution
* Edge probabilities in an SBM are modeled using Bernoulli distributions due to the binary nature (either an edge exists or it doesn't)
* Given a pair of nodes, the probability of an edge existing between them follows a Bernoulli distribution

* p is the probability of success
* x is the binary outcome variable (0 or 1, representing the absence or presence of an edge)
* if a random number drawn from this distribution is less than or equal to p (p_in or p_out), an edge is considered to exist between the nodes, otherwise, there is no edge

##### Edge modeling
* Edge probabilities in SBM are typically estimated from the observed network data using MLE or Bayesian inference
* These probabilities capture the underlying structure of the network (community structures, connectivity patterns, relationship strengths)

### Why SBM?
While we were at the stage of processing customer data, we believed that each client within a company, even though they don't know each other, they may likely give off the same churn behavior based on some features on our data.

* Each block is a group of customers with similar behaviors, preferences, or interaction patterns. Customers within the same block are more likely to exhibit similar churn behaviors compared to those in different blocks
* Customers who are more connected within the network might be less likely to churn due to stronger tie or social influence, word-of-mouth effect, etc.
* Customers who frequently use complementary products within the same network might exhibit lower churn rates, due to increased product stickiness and statisfaction.

### Determine the optimal number of blocks (block size), the probability of an edge existing between nodes within each cluster, and the probability of an edge existing between clusters
* Denote probability of an edge existing between nodes within each cluster: p_in
* Denote probability of an edge existing between clusters: p_out

To determine the optimal number of those, we follow the following steps: 
1. Perform PCA for dimensionality reduction over our dataset so that we can extract the most important features
2. Perform clustering (K-means, K-modes, etc) to cluster the dimensionality reduced dataset
3. Visualize and get the number of clusters, and use the number for the initial number of blocks of our SBM
4. Determine the optimal p_in by similarity metric and p_out by dissimilarity metric: Within-cluster similarity can provide insights to strength of connections within blocks. Between-clusters dissimilarity can provide insights to separations between blocks.
5. We can get the optimal p_in and p_out by MLE by: Calculate the average similarity (edge) within each cluster and assign to p_in. Calculate the average dissimilarity (edge) between clusters and assign to p_out.

Decoding `investment_objective` data:

Similar to `risk_tolerance`, for `investment_objective`, the alphanumeric digit indicates the type of security associated with each client, and the number next to each type indicates the probability of having that type of security for each client.

#### Determine the feature importance for: Features to determine the connection between blocks, and features to determine the connection between nodes within each block
* More important features will be used to determine the segmentation of blocks
* Less important features will be used to determine the segmentation of nodes within each block

#### Strategy development
* If certain blocks of customers are identified as high risk for churn, specific strategies can be designed for these blocks
* Understanding the interaction patterns can help in devising referral programs, **cross-selling strategies** and personalized marketing campaigns that leverage the network effects within and across customer blocks